### Ref: Kernels

https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

https://www.kaggle.com/artgor/nn-baseline

https://www.kaggle.com/ogrellier/user-level-lightgbm-lb-1-4480

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from sklearn.preprocessing import LabelEncoder

In [ ]:
print(os.listdir("../input"))


In [ ]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, 
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
%%time
df_train_raw = load_df('../input/train.csv')

In [ ]:
%%time
df_test_raw = load_df('../input/test.csv')

###  keep _raw for resuming point

In [ ]:
%%time
df_train = df_train_raw.copy()

In [ ]:
%%time
df_test = df_test_raw.copy()

### apply adjustments to both train and test datasets

In [ ]:
def applyEdits(df):
    # fill NA's
    df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True)
    df['trafficSource.isTrueDirect'].fillna(False, inplace=True)

    l_fillwithzero = ['totals.bounces','totals.newVisits','totals.pageviews',
    'trafficSource.adContent','trafficSource.keyword','trafficSource.adwordsClickInfo.adNetworkType',
    'trafficSource.adwordsClickInfo.adNetworkType','trafficSource.adwordsClickInfo.gclId',
    'trafficSource.adwordsClickInfo.gclId','trafficSource.adwordsClickInfo.page',
    'trafficSource.adwordsClickInfo.slot']
    for x in l_fillwithzero:
        df[x] = df[x].fillna(0)

    # add / edit columns
    df['browser_category'] = df['device.browser'] + '_' + df['device.deviceCategory']
    df['browser_operatingSystem'] = df['device.browser'] + '_' + df['device.operatingSystem']
    df['source_country'] = df['trafficSource.source'] + '_' + df['geoNetwork.country']                                                                   

    
    df['dummy'] = 1
    df['user_cumcnt_per_day'] = (df[['fullVisitorId','date', 'dummy']].groupby(['fullVisitorId','date'])['dummy'].cumcount()+1)
    df['user_sum_per_day'] = df[['fullVisitorId','date', 'dummy']].groupby(['fullVisitorId','date'])['dummy'].transform(sum)
    df['user_cumcnt_sum_ratio_per_day'] = df['user_cumcnt_per_day'] / df['user_sum_per_day'] 
    df.drop('dummy', axis=1, inplace=True)

    return df

In [ ]:
%%time
df_train = applyEdits(df_train)

In [ ]:
%%time
df_test = applyEdits(df_test)

### apply only for train dataset

In [ ]:
# to float and log
df_train['totals.transactionRevenue'] = df_train['totals.transactionRevenue'].astype(float)
df_train['totals.transactionRevenue'] = np.log1p(df_train['totals.transactionRevenue'].fillna(0))

### delete columns with no valid info

In [ ]:
%%time
cols_to_drop = [col for col in df_train.columns if df_train[col].nunique() == 1]
df_train.drop(cols_to_drop, axis=1, inplace=True)
df_test.drop([col for col in cols_to_drop if col in df_test.columns], axis=1, inplace=True)


### clean up date column

In [ ]:
def cleanDate(df):
    df['date'] = pd.to_datetime(df['date'].apply(lambda x: str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:]))
    return df


df_train = cleanDate(df_train)
df_test = cleanDate(df_test)

In [ ]:
%%time

def parseDates(df):
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday
    df['weekofyear'] = df['date'].dt.weekofyear
    
    return df


df_train = parseDates(df_train)
df_test = parseDates(df_test)


In [ ]:
%%time
def strToNum(df):
    l = [ 'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews',]
    for x in l:
        df[x] = df[x].values.astype(np.int64)
    return df

df_train = strToNum(df_train)
df_test = strToNum(df_test)

In [ ]:
no_use = ['date', 'fullVisitorId', 'sessionId', 'visitId', 'visitStartTime', 'totals.transactionRevenue', 'trafficSource.referralPath']

cat_cols = [x for x in df_train.columns if x not in no_use and type(df_train[x][0]) == str]

num_cols = [x for x in df_train.columns if x not in no_use and x not in cat_cols and type(df_train[x][0]) != str and type(df_train[x][0]) != bool]


### Encode category strings to number labels ** use same number for same category accross train & test datasets

In [ ]:
%%time
max_values = {}
for col in cat_cols:
    lbl = LabelEncoder()
    lbl.fit(list(df_train[col].values.astype('str')) + list(df_test[col].values.astype('str')))
    df_train[col] = lbl.transform(list(df_train[col].values.astype('str')))
    df_test[col] = lbl.transform(list(df_test[col].values.astype('str')))
    max_values[col] = max(df_train[col].max(), df_test[col].max())  + 2


In [77]:
%%time
def boolToNum(df):
    l = ['trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect' ]
    for x in l:
        df[x] = df[x].values.astype(np.int64) * 1
    return df
df_train = boolToNum(df_train)
df_test = boolToNum(df_test)

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 82.4 ms


In [81]:
df_train.drop('trafficSource.referralPath', axis=1, inplace=True)
df_test.drop('trafficSource.referralPath', axis=1, inplace=True)


## -------------------------------------------------------------for reseting ---------------------------------------------------------------

In [83]:
%%time
#export
df_train.to_csv('train_wip.csv')
df_test.to_csv('test_wip.csv')

#save
#df_train_forReset = df_train.copy()
#df_test_forReset = df_test.copy()

#load
#df_train = df_train_forReset.copy()
#df_test = df_test_forReset.copy()


CPU times: user 1min 11s, sys: 708 ms, total: 1min 11s
Wall time: 1min 11s


In [84]:

def aggregate_by_users(df, cat_cols):
    aggs = {
        'date': ['min', 'max'],
        'totals.transactionRevenue': ['sum', 'size'],
        'totals.hits': ['sum', 'min', 'max', 'mean', 'median'],
        'totals.pageviews': ['sum', 'min', 'max', 'mean', 'median'],
        'totals.bounces': ['sum', 'mean', 'median'],
        'totals.newVisits': ['sum', 'mean', 'median']
    }

    for f in cat_cols + ['weekday', 'day', 'month', 'weekofyear']:
        aggs[f] = ['min', 'max', 'mean', 'median', 'var', 'std']

    users = df.groupby('fullVisitorId').agg(aggs)

    new_columns = [
        k + '_' + agg for k in aggs.keys() for agg in aggs[k]
    ]
    users.columns = new_columns

    users['date_diff'] = (users.date_max - users.date_min).astype(np.int64) // (24 * 3600 * 1e9)
    
    return users

In [93]:
%%time
df_user_train = aggregate_by_users(df_train, cat_cols)

df_test['totals.transactionRevenue'] = 0
df_user_test = aggregate_by_users(df_test, cat_cols)



CPU times: user 16 s, sys: 1.98 s, total: 17.9 s
Wall time: 18.2 s


## save aggregated csv

In [95]:
%%time
#export
df_user_train.to_csv('user_train_wip.csv')
df_user_test.to_csv('user_test_wip.csv')

CPU times: user 3min, sys: 1.3 s, total: 3min 1s
Wall time: 3min 1s


# ------------------------ work in progress --------------------

## model

In [1]:
%%time

from sklearn.model_selection import train_test_split

df_y = df_user_train['totals.transactionRevenue_sum']
df_x = df_user_train.drop(['date_min', 'date_max', 'totals.transactionRevenue_sum'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25)



NameError: name 'df_user_train' is not defined

In [ ]:
lgb_params = {
    'learning_rate': 0.03,
    'n_estimators': 2000,
    'num_leaves': 128,
    'subsample': 0.2217,
    'colsample_bytree': 0.6810,
    'min_split_gain': np.power(10.0, -4.9380),
    'reg_alpha': np.power(10.0, -3.2454),
    'reg_lambda': np.power(10.0, -4.8571),
    'min_child_weight': np.power(10.0, 2),
    'silent': True
}

for fold_, (trn_, val_) in enumerate(folds.split(trn_users)):
    model = lgb.LGBMRegressor(**lgb_params)

    model.fit(
        trn_users.iloc[trn_], trn_y.iloc[trn_],
        eval_set=[(trn_users.iloc[trn_], trn_y.iloc[trn_]),
                  (trn_users.iloc[val_], trn_y.iloc[val_])],
        eval_metric='rmse',
        early_stopping_rounds=100,
        verbose=0
    )

    oof_preds[val_] = model.predict(trn_users.iloc[val_])
    curr_sub_preds = model.predict(sub_users)
    curr_sub_preds[curr_sub_preds < 0] = 0
    sub_preds += curr_sub_preds / folds.n_splits
    #     preds[preds <0] = 0

    logger.info('Fold %d RMSE (raw output) : %.5f' % (fold_ + 1, rmse(trn_y.iloc[val_], oof_preds[val_])))
    oof_preds[oof_preds < 0] = 0
    oof_scores.append(rmse(trn_y.iloc[val_], oof_preds[val_]))
    logger.info('Fold %d RMSE : %.5f' % (fold_ + 1, oof_scores[-1]))

logger.info('Full OOF RMSE (zero clipped): %.5f +/- %.5f' % (rmse(trn_y, oof_preds), float(np.std(oof_scores))))

# Stay in logs for submission
sub_users['PredictedLogRevenue'] = sub_preds
sub_users[['PredictedLogRevenue']].to_csv("simple_lgb.csv", index=True)

logger.info('Submission data shape : {}'.format(sub_users[['PredictedLogRevenue']].shape))

hist, bin_edges = np.histogram(np.hstack((oof_preds, sub_preds)), bins=25)
plt.figure(figsize=(12, 7))
plt.title('Distributions of OOF and TEST predictions', fontsize=15, fontweight='bold')
plt.hist(oof_preds, label='OOF predictions', alpha=.6, bins=bin_edges, density=True, log=True)
plt.hist(sub_preds, label='TEST predictions', alpha=.6, bins=bin_edges, density=True, log=True)
plt.legend()
plt.savefig('distributions.png')
